<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# [Scénario d'Apprentissage Statistique](https://github.com/wikistat/Apprentissage)

# Modélisation de données d'enquête en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a>:  prévision du seuil de revenu 

## Introduction
Des données publiques disponibles sur le site [UCI repository](http://archive.ics.uci.edu/ml/) sont extraites de la base de données issue du recensement réalisé aux Etats Unis en 1994. Ces données son largement utilisées et font référence comme outil de *benchmark* pour comparer les performances de méthodes d’apprentissage ou modélisation statistique. L’objectif est  de prévoir la variable binaire « revenu annuel » (`income`) supérieur ou inférieur à 50k$. Il ne s’agit pas encore de données massives mais 32561 individus sont décrits par les 14 variables du tableau ci-dessous :

Num| Libellé |	Ensemble de valeurs
-|--|--|--
1|`Age`|	real
2|	`workClass`|	Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked
3|	`fnlwgt`|	real
4|	`education`|	Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool
5|	`educNum`|	integer
6|	`mariStat`|	Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse
7|	`occup`|	Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces
8|	`relationship`|	Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried
9|	`origEthn`|	White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black
10|	`sex`|	Female, Male
11|	`capitalGain`|	real  
12|	`capitalLoss`|	real
13|	`hoursWeek`|	real
14|	`nativCountry`|	United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands
15|	`income`|		incHigh (>50K), incLow (<=50K)

Une première étape permettant de vérifier, sélectionner, recoder les données, a permis de construire un fichier de type `.csv` qui est utilisé dans ce calepin.


**Répondre aux questions en s'aidant des résultats des exécutions**.

## 1 Préparation des données
### 1.1 Lecture  

Charger les [données](https://www.math.univ-toulouse.fr/~besse/Wikistat/data/adultCensus.csv)  dans le répertoire courant (`path=""`) ou exécuter directement la cellule. 

In [ ]:
%matplotlib inline
# Importations 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
path=""  # si les données sont déjà dans le répertoire courant
#path="http://www.math.univ-toulouse.fr/~besse/Wikistat/data/"
datBase=pd.read_csv(path+'adultCensus.csv')
datBase.head()

Repérer les variables quantitatives, qualitatives. Certaines (`age`, `hoursWeek`) sont présentes sous les deux types. Beaucoup de modalités on déjà été regroupées (voir le programme) certaines variables sont rendues qualitatives (`capitalLoss` ou `Gain`) et transformées par une fonction *log*. La variable à modéliser est `income`.

### 1.2 Exploration élémentaire et vérifications

In [ ]:
datBase.dtypes

In [ ]:
# Beaucoup de variables qualitatives
datBase.describe()

In [ ]:
# pas de données manquantes
datBase.count()

**Q** Que dire de la distribution de la variable `age`, de celle `income` ?

In [ ]:
datBase["age"].hist()
plt.show()

In [ ]:
datBase["fnlwgt"].hist()
plt.show()

In [ ]:
datBase["income"].value_counts()

In [ ]:
datBase["relationship"].value_counts()

In [ ]:
datBase.plot(kind="scatter",x="age",y="educNum")
plt.show()

**Q** Que dire des liaisons : `age x hoursWeek`, `age x income`, `sex x income` ?

In [ ]:
datBase.plot(kind="scatter",x="hoursWeek",y="age")
plt.show()

In [ ]:
datBase.boxplot(column="age",by="income")
plt.show()

**Q** La variable `fnlwgt` (Final sampling weight:  Inverse of sampling fraction adjusted for non-response and over or under sampling of particular groups) est assez obscure.  Que dire de sa liaison avec la variable cible? Elle est supprimée par la suite

In [ ]:
datBase.boxplot(column="fnlwgt",by="income")
plt.show()

In [ ]:
# Mosaic plot
from statsmodels.graphics.mosaicplot import mosaic
mosaic(datBase,["income","sex"])
plt.show()

### 1.3 Variables indicatrices
**Q** Pourquoi l’introduction de dummy variables ? Pour quelles méthodes cela serait-il aussi nécessaire en R?

In [ ]:
datBaseDum=pd.get_dummies(datBase[["workClass","education","mariStat",
    "occup","relationship","origEthn","sex","capitalGain","capitalLoss",
    "nativCountry","ageQ","hoursWeekQ"]])
datBaseDum.head()

In [ ]:
X = datBase[["age","educNum","hoursWeek","LcapitalGain",
             "LcapitalLoss","income"]].join(datBaseDum)
X.head()

In [ ]:
# Variable cible
Y=datBase["income"]
# Variables prédictives
X=X.drop(["income"],axis=1)

### 1.3  Extraction des échantillons
**Q** Quel est l’objectif de cette cellule ? Justifier la nécessité de ce traitement.

In [ ]:
from sklearn.model_selection import train_test_split  
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=3000,random_state=11)

## 2 Méthodes de modélisation 
L'algorithme des *k* plus proches voisins fourni des réultats assez catastrophiques sur ces données, il n'est pas repris. Est-ce dû au nombre important d'indicatrices dans les variables explicatives? A confirmer sur d'autres exemples.
### 2.1 Régression logistique
**Q** Commenter  les options de la commande  `GridSearchCV`. A quoi sert `param` ?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import time
tps0=time.clock()
# Optimisation du paramètre de pénalisation
# grille de valeurs
param=[{"C":[0.1, 0.15,0.2,0.25,0.3]}]
logit = GridSearchCV(LogisticRegression(penalty="l1"), param,cv=10,n_jobs=-1)
logitOpt=logit.fit(X_train, Y_train)  # GridSearchCV est lui même un estimateur
# paramètre optimal
logitOpt.best_params_["C"]
tps1=(time.clock()-tps0)/60
print("Temps logit = %f, Meilleur taux = %f, Meilleur paramètre = %s" % (tps1,
                              1.-logitOpt.best_score_,logitOpt.best_params_))

In [ ]:
# erreur sur l'échantillon test
1-logitOpt.score(X_test, Y_test)

In [ ]:
# Prévision
y_chap = logitOpt.predict(X_test)
# matrice de confusion
table=pd.crosstab(y_chap,Y_test)
print(table)

**Q** La matrice de confusion n’est pas symétrique. Quelle pourrait en être la raison ?

**Q** Quels algorithmes pourraient être exécutés en R pour la régression logistique? Quelles informations complémentaires en tirer?

In [ ]:
# Coefficients
LogisticRegression(penalty="l1",C=logitOpt.best_params_['C']).fit(X_train, Y_train).coef_

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
model = LogisticRegressionCV(Cs=10,cv=5, penalty="l1",
        n_jobs=-1,random_state=13,solver="liblinear").fit(X_train,Y_train)
m_log_alphas = -np.log10(model.Cs_)

In [ ]:
plt.figure()
plt.plot(m_log_alphas, model.scores_['incLow'].T, ':')
plt.plot(m_log_alphas, model.scores_['incLow'].T.mean(axis=-1), 'k',
         label='precision moyenne', linewidth=2)
plt.axvline(-np.log10(model.C_), linestyle='--', color='k',
            label='Cs: optimal par VC')
plt.legend()
plt.xlabel('-log(CS)')
plt.ylabel(u'Précision')
plt.title(u'Précision de chaque validation')
plt.show()

**Q** Que représente le graphique ? Comment l’interpréter ?

### 2.2 Arbre binaire de discrimination

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tps0=time.clock()
# Optimisation de la profondeur de l'arbre
param=[{"max_depth":list(range(2,10))}]
tree= GridSearchCV(DecisionTreeClassifier(),param,cv=10,n_jobs=-1)
treeOpt=tree.fit(X_train, Y_train)
# paramètre optimal
tps1=(time.clock()-tps0)/60
print("Temps arbre = %f, Meilleur taux = %f, Meilleur paramètre = %s" % (tps1,
                             1. - treeOpt.best_score_,treeOpt.best_params_))

In [ ]:
# Estimation de l'erreur de prévision
1-treeOpt.score(X_test,Y_test)

In [ ]:
# prévision de l'échantillon test
y_chap = treeOpt.predict(X_test)
# matrice de confusion
table=pd.crosstab(y_chap,Y_test)
print(table)

In [ ]:
treeOpt.best_params_['max_depth']

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
import pydot
treeG=DecisionTreeClassifier(max_depth=treeOpt.best_params_['max_depth'])
treeG.fit(X_train,Y_train)
dot_data = StringIO() 
export_graphviz(treeG, out_file=dot_data) 
graph=pydot.graph_from_dot_data(dot_data.getvalue()) 
graph.write_png("treeOpt.png")  

In [ ]:
from IPython.display import Image
Image(filename='treeOpt.png')

**Q** Quelle est l’insuffisance de l’implémentation des arbres de décision dans Scikit-learn  par rapport à celle de rpart de R ? Que dire de l’arbre?

### 2.3 Perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler  
# L'algorithme ds réseaux de neurones nécessite éventuellement une normalisation 
# des variables explicatives avec les commandes ci-dessous
scaler = StandardScaler()  
scaler.fit(X_train)  
Xnet_train = scaler.transform(X_train)  
# Meme transformation sur le test
Xnet_test = scaler.transform(X_test)

In [ ]:
tps0=time.clock()
param_grid=[{"hidden_layer_sizes":list([(5,),(6,),(7,),(8,)])}]
nnet= GridSearchCV(MLPClassifier(max_iter=500),param_grid,cv=10,n_jobs=-1)
nnetOpt=nnet.fit(Xnet_train, Y_train)
# paramètre optimal
tps1=(time.clock()-tps0)/60
print("Temps perceptron = %f, Meilleur taux = %f, Meilleur paramètre = %s" % (tps1,
                                    1. - nnetOpt.best_score_,nnetOpt.best_params_))

**Q** Quelle stratégie d’optimisation est adoptée ? Quelle autre pourrait l’être? Quel réseau pourrait également être pris en compte? Quelles sont les fonctions d’activation des neurones?

In [ ]:
# Estimation de l'erreur de prévision sur le test
1-nnetOpt.score(Xnet_test,Y_test)

In [ ]:
# prévision de l'échantillon test
y_chap = nnetOpt.predict(Xnet_test)
# matrice de confusion
table=pd.crosstab(y_chap,Y_test)
print(table)

### 2.4 Forêts aléatoires
**Q** Commenter les choix de tous les paramètres.

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
# définition des paramètres
forest = RandomForestClassifier(n_estimators=500, 
   criterion='gini', max_depth=None,
   min_samples_split=2, min_samples_leaf=1, 
   max_features='auto', max_leaf_nodes=None,
   bootstrap=True, oob_score=True)
# apprentissage
rfFit = forest.fit(X_train,Y_train)
print(1-rfFit.oob_score_)

In [ ]:
# erreur de prévision sur le test
1-rfFit.score(X_test,Y_test)

In [ ]:
# optimisation du paramètre
tps0=time.clock()
param=[{"max_features":list(range(2,10,1))}]
rf= GridSearchCV(RandomForestClassifier(n_estimators=100),param,cv=10,n_jobs=-1)
rfOpt=rf.fit(X_train, Y_train)
# paramètre optimal
tps1=(time.clock()-tps0)/60
print("Temps r forest = %f, Meilleur taux = %f, Meilleur paramètre = %s" % (tps1,
                                    1. - rfOpt.best_score_,rfOpt.best_params_))

In [ ]:
# erreur de prévision sur le test
1-rfOpt.score(X_test,Y_test)

In [ ]:
# prévision
y_chap = rfFit.predict(X_test)
# matrice de confusion
table=pd.crosstab(y_chap,Y_test)
print(table)

In [ ]:
rf= RandomForestClassifier(n_estimators=100,max_features=2)
rfFit=rf.fit(X_train, Y_train)
# Importance décroissante des variables
importances = rfFit.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(20):
    print(X_train.columns[indices[f]], importances[indices[f]])

In [ ]:
# Graphe des importances
plt.figure()
plt.title("Importances des variables")
plt.bar(range(X_train.shape[1]), importances[indices])
plt.xticks(range(X_train.shape[1]), indices)
plt.xlim([-1, X_train.shape[1]])
plt.show()

**Q** Comment est obtenu le graphique ? Quelle importance ? Comment interpréter ces résultats ?

### 2.5 Gradient boosting
**Q** Pourquoi pas de paramètre `njobs=-1`? 
**Q** En plus de celui `optimiser, quels sont les 2 principaux paramètres cet algorithme laissés par défaut ?

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
tps0=time.clock()
param=[{"n_estimators":[100, 150, 200, 250]}]
gbm= GridSearchCV(GradientBoostingClassifier(),param,cv=10)
gbmOpt=gbm.fit(X_train, Y_train)
# paramètre optimal
tps1=(time.clock()-tps0)/60
print("Temps boosting = %f, Meilleur taux = %f, Meilleur paramètre = %s" % (tps1,
                                        1. - gbmOpt.best_score_,gbmOpt.best_params_))

In [ ]:
# erreur de prévision sur le test
1-gbmOpt.score(X_test,Y_test)

In [ ]:
# prévision de l'échantillon test
y_chap = gbmOpt.predict(X_test)
# matrice de confusion
table=pd.crosstab(y_chap,Y_test)
print(table)

## 3 Comparaison des méthodes

### 3.1 Courbes ROC
**Q** En cohérence avec les résultats précédents, quelle est la courbe la plus au dessus des autres? Commenter ce graphique, que signifie AUC ?

In [ ]:
from sklearn.metrics import roc_curve
listMethod=[["GBM",gbmOpt],["RF",rfOpt],["NN",nnetOpt],["Tree",treeOpt],["Logit",logitOpt]]

In [ ]:
# Modèles tous réajustés sur les mêmes variables centrées réduites
for method in enumerate(listMethod):
    probas_ = method[1][1].fit(Xnet_train, Y_train).predict_proba(Xnet_test)
    fpr, tpr, thresholds = roc_curve(Y_test,probas_[:,1], pos_label="incLow")
    plt.plot(fpr, tpr, lw=1,label="%s"%method[1][0]),
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.legend(loc="best")
plt.show()

### 3.2 Validation croisée *Monte Carlo*
**Q** Quelle différence entre la validation croisée Monte Carlo et la V-fold cross validation?

**Q** Les variables sont « standardisées ». Pourquoi ? Est-ce important et pour quelles méthodes? Commenter les résultats. Quelle méthode choisir ?

In [ ]:
from sklearn.utils import check_random_state
import time
tps0=time.clock()
check_random_state(11)
# définition des estimateurs
logit= LogisticRegression(penalty="l1")
tree = DecisionTreeClassifier()
nnet = MLPClassifier(max_iter=400)
rf   = RandomForestClassifier(n_estimators=200)
gbm  = GradientBoostingClassifier()
# Nombre d'itérations
B=10 # pour tester le programme, mettre plutôt B=3
# définition des grilles de paramètres
listMethGrid=[[gbm,{"n_estimators":[100, 150, 200, 250]}],
    [rf,{"max_features":list(range(6,10))}],
    [nnet,{"hidden_layer_sizes":list([(5,),(6,),(7,),(8,)])}],
    [tree,{"max_depth":list(range(2,10))}],
    [logit,{"C":[0.1, 0.15,0.2,0.25,0.3]}]]
# Initialisation à 0 
arrayErreur=np.empty((B,5))
for i in range(B):   # itérations sur B échantillons test
    # extraction apprentissage et test
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=1000)
    scaler = StandardScaler()  
    scaler.fit(X_train)  
    Xnet_train = scaler.transform(X_train)  
    # Meme transformation sur le test
    Xnet_test = scaler.transform(X_test)
    # optimisation de chaque méthode et calcul de l'erreur sur le test
    for j,(method, grid_list) in enumerate(listMethGrid):
        methodGrid=GridSearchCV(method,grid_list,cv=10,n_jobs=-1).fit(X_train, Y_train)
        methodOpt = methodGrid.best_estimator_
        methFit=methodOpt.fit(Xnet_train, Y_train)
        arrayErreur[i,j]=1-methFit.score(Xnet_test,Y_test)
tps1=time.clock()

In [ ]:
dataframeErreur=pd.DataFrame(arrayErreur,columns=["GBM","RF","NN","Tree","Logit"])
print("Temps execution en mn :",(tps1 - tps0)/60)

In [ ]:
dataframeErreur[["GBM","RF","NN","Tree","Logit"]].boxplot(return_type='dict')
plt.show()

In [ ]:
# Moyennes
dataframeErreur.mean()

**Q** Les SVM ne font pas partie de la comparaison à cause de temps rédhibitoires d’exécution. A quoi est-ce dû ? Commenter les temps d’exécution des différentes étapes. Quelle autre package pourrait être utilisé pour la section 2.5 ? Pourquoi ?